# Cycloid Fitting

### Simplified to reduce notebook size

## Load Cyloid Data Points and Bezier Curves

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import curves.bezier as bezier
import curves.fitCurves as fit
import StressTools as tools
import utils
import fitting
import cycloidData
from scipy import stats
from os import path
from datetime import datetime

interior = utils.import_interior('interior1')

TOLERANCE = 1

cycloids = cycloidData.load_all_cycloids()
highResCycloids = cycloidData.load_all_cycloids(points_per_curve=1000)

min_vals = np.array([0, 0.1, 0])
max_vals = np.array([360, 1, 360])
constraints = [
    dict(wrapValue=True, minValue=1e-8, maxValue=1),
    dict(minValue=1e-8, maxValue=1, unstick=True),
    dict(wrapValue=True, minValue=0, maxValue=1)
]

tight_obliquity_contstraints = [
        dict(wrapValue=True, minValue=1e-8, maxValue=1),
        dict(minValue=0.62, maxValue=0.82, unstick=True),
        dict(wrapValue=True, minValue=0, maxValue=1)
    ]

def getConstraints(obliquity, longitude_max=1):
    return [
        dict(wrapValue=True, minValue=1e-8, maxValue=1),
        dict(minValue=obliquity, maxValue=obliquity, unstick=False),
        dict(wrapValue=True, minValue=0, maxValue=longitude_max)
    ]

def get_longitude_only_constraints(obliquity, phase):
    return [
        dict(minValue=phase, maxValue=phase, unstick=False),
        dict(minValue=obliquity, maxValue=obliquity, unstick=False),
        dict(wrapValue=True, minValue=0, maxValue=1)
    ]



### Helper Functions

In [2]:
def logmessage(msg):
    now = datetime.now()
    timestamp = now.strftime("%H:%M:%S")
    
    print(f'[{timestamp}] {msg}')
    

def translate_params(params, minVals, maxVals):
    if len(params) == 3:
        variables = params * (max_vals - min_vals) + min_vals # denormalize
    else:
        variables = params * (max_vals[0:2:] - min_vals[0:2:]) + min_vals[0:2:]

    return variables

def setChartXLimit(points, plt):
    BUFFER_PERCENT = 0.025

    first = points['lon'].max()
    last = points['lon'].min()

    buffer = (first - last) * BUFFER_PERCENT

    plt.xlim(first + buffer, last - buffer)


def check_fit(params, minVals, maxVals, curve, interior, tolerance=0.25, title='',
              verbose=True, path='./output/stressfield.csv.gz'):

    if len(params) == 3:
        variables = params * (max_vals - min_vals) + min_vals # denormalize
    else:
        variables = params * (max_vals[0:2:] - min_vals[0:2:]) + min_vals[0:2:]

    data, loss = fitting.match_stresses(curve,
                                        variables,
                                        interior,
                                        save_stress_field=True,
                                        path=path)
    if verbose:
        plt.figure()
        plt.title(f'{title} - Orientation Match')
        fit_points = data.loc[data['deltaHeading'] < tolerance].copy()

        if len(variables) >= 3:
            fit_points['lon'] = fit_points['lon'] - variables[2]

        plt.plot(curve['lon'], curve['lat'])
        setChartXLimit(curve, plt)

        plt.scatter(fit_points['lon'], fit_points['lat'], alpha=0.3, color='green')

        plt.figure()
        plt.title(f'{title} - Stress Magnitude')
        plt.scatter(fit_points['pointNumber'], fit_points['stress'])

        print(np.array(variables))

    return data

def plot_time(data):
    timeData = data.copy()
    time = np.array(timeData['time'])
    time[time < 180] = time[time < 180] + 360
    timeData['time'] = time

    plt.figure()
    plt.scatter(timeData['pointNumber'], timeData['time'],s=1)
    plt.title("Time")

def analyze_fit(opt, curve, tolerance, name=''):
    plt.figure()
    plt.title('Optimizer Loss Values')
    plt.plot(opt[0])

    display(opt[1])
    display(opt[2])

    params = opt[1]['parameters']
    bestCase = check_fit(params, min_vals, max_vals, curve, interior, tolerance=tolerance,
                         title=f'{name} Best Fit')

    params = opt[2]['parameters']
    finalCase = check_fit(params, min_vals, max_vals, curve, interior, tolerance=tolerance,
                         title=f'{name} Final Fit')


    # Plot time progression
    plot_time(bestCase)

    return bestCase, finalCase

def analyze_params(params, curve, tolerance, name='', verbose=True, folder='./output/', suffix=''):
    fieldPath = path.join(folder, f'{name}{suffix}StressField.csv.gz')
    results = check_fit(params, min_vals, max_vals, curve, interior, tolerance=tolerance,
                       title=f'{name} Highest Probability Fit', verbose=verbose, path=fieldPath)

    if verbose:
        plot_time(results)

    return results

def direct_fit(curve, tolerance, show_plots=True, params=[0, 0, 0]):
    data, loss = fitting.match_stresses(curve, params, interior)
    fit_points = data.loc[data['deltaHeading'] < tolerance].copy()

    if show_plots:
        plt.figure()
        plt.plot(curve['lon'], curve['lat'])
        setChartXLimit(curve, plt)

        plt.scatter(fit_points['lon'], fit_points['lat'], alpha=0.3, color='green')
        plt.figure()
        plt.scatter(fit_points['pointNumber'], fit_points['stress'])

        plot_time(data)

    return data, loss

def process_cycloid(curve,
                    name,
                    paramCount = 2,
                    analysisCurve=None,
                    folder='./output/',
                    iterations=500,
                    constraints=tight_obliquity_contstraints,
                    verbose=True):

    logmessage(f'Processing Cycloid: {name}')
    plt.close('all')
    numParams = paramCount
    start_params = [np.random.rand() for iter in range(numParams)]
    start_params[1] = 0.67

    optimizer = fitting.Adam(alpha=0.05)

    opt = optimizer.minimize(
        fitting.test_stress_parameters,
        curve,
        start_params,
        interior,
        constraints=constraints,
        max_iterations=iterations,
        verbose=verbose,
        batch_size=16
    )

    params = fitting.find_best_parameters(opt)
    fullCurve = analysisCurve if analysisCurve is not None else curve
    
    logmessage(f'Analyzing optimization parameters for {name}')
    bestFit = analyze_params(np.array(params), fullCurve, 0.25, name, verbose, folder)

    cols = ['loss', 'phase', 'obliquity']
    df = pd.DataFrame(opt[3], columns=cols).copy()

    df['phase'] = df['phase'] * (max_vals[0] - min_vals[0]) + min_vals[0]
    df['obliquity'] = df['obliquity'] * (max_vals[1] - min_vals[1]) + min_vals[1]

    df.to_csv(f'{folder}{name}Fits.csv.gz', index=False, encoding='utf-8', compression='gzip')
    bestFit.to_csv(f'{folder}{name}BestFit.csv.gz', index=False, encoding='utf-8', compression='gzip')
    
def process_cycloid_by_name(name, folder='./output/', iterations=5000, constraints=constraints, verbose=False):
    process_cycloid(cycloids[name].curve,
                    name,
#                     analysisCurve=highResCycloids[name].curve,
                    folder=folder,
                    iterations=iterations,
                    constraints=constraints,
                    verbose=verbose)

def process_cycloid_top_fits(curve,
                    name,
                    paramCount = 2,
                    analysisCurve=None,
                    folder='./output/',
                    iterations=500,
                    constraints=constraints,
                    verbose=True,
                    number_of_fits=5):

    numParams = paramCount
    start_params = [np.random.rand() for iter in range(numParams)]
    start_params[1] = 0.67

    optimizer = fitting.Adam(alpha=0.05)

    opt = optimizer.minimize(
        fitting.test_stress_parameters,
        curve,
        start_params,
        interior,
        constraints=constraints,
        max_iterations=iterations,
        verbose=verbose,
        batch_size=16
    )

    cols = ['loss', 'phase', 'obliquity']
    if paramCount == 3:
        cols.append('longitude')
    fitFrame = pd.DataFrame(opt[3],
                            columns=cols).sort_values('loss')[0:number_of_fits]
    fitFrame['FitNumber'] = range(1, number_of_fits + 1)

    fullCurve = analysisCurve if analysisCurve is not None else curve

    for fit in fitFrame.itertuples():
        params = [fit.phase, fit.obliquity]
        if paramCount == 3:
            params.append(fit.longitude)
        bestFit = analyze_params(np.array(params),
                                 fullCurve,
                                 0.25,
                                 name,
                                 verbose,
                                 folder,
                                 suffix=fit.FitNumber)
        bestFit.to_csv(f'{folder}{name}BestFit{fit.FitNumber}.csv.gz',
                       index=False, encoding='utf-8', compression='gzip')

    df = pd.DataFrame(opt[3], columns=cols).copy()

    df['phase'] = df['phase'] * (max_vals[0] - min_vals[0]) + min_vals[0]
    df['obliquity'] = df['obliquity'] * (max_vals[1] - min_vals[1]) + min_vals[1]
    if paramCount == 3:
        df['longitude'] = df['longitude'] * (max_vals[2] - min_vals[2]) + min_vals[2]

    df.to_csv(f'{folder}{name}Fits.csv.gz', index=False, encoding='utf-8', compression='gzip')

    
def process_cycloid_by_name(name, folder='./output/', iterations=3000, constraints=constraints, verbose=False):
    process_cycloid(cycloids[name].curve,
                    name,
                    analysisCurve=highResCycloids[name].curve,
                    folder=folder,
                    iterations=iterations,
                    constraints=constraints,
                    verbose=verbose)

## Generate Cyloid Data in Full Obliquity Range

In [3]:
for key in cycloids:
    process_cycloid_by_name(key, 
                            iterations=2000,
                            folder='./output/fullObliquityRange/',
                            verbose=False)

[21:04:17] Processing Cycloid: alex
Iteration 150/2000 -- Loss Output: 0.6959423023664197 -- Moving Avg Loss: 0.2432922936852552
	Parameters used: [0.66490939 0.73141345]
Iteration 300/2000 -- Loss Output: 0.046143822698847065 -- Moving Avg Loss: 0.05700536166836435
	Parameters used: [0.72061004 0.78717214]
Iteration 450/2000 -- Loss Output: 0.0545609300259998 -- Moving Avg Loss: 0.05726755292427826
	Parameters used: [0.75612898 0.82272809]
Iteration 600/2000 -- Loss Output: 0.07157818170460832 -- Moving Avg Loss: 0.07021029708137697
	Parameters used: [0.83442248 0.90110314]
Iteration 750/2000 -- Loss Output: 0.0952347677821205 -- Moving Avg Loss: 0.09661069222390205
	Parameters used: [0.89342902 0.96017114]
Iteration 900/2000 -- Loss Output: 0.08962440163247792 -- Moving Avg Loss: 0.07005767921882433
	Parameters used: [0.85838712 0.99875183]
Iteration 1050/2000 -- Loss Output: 11.022894006922217 -- Moving Avg Loss: 4.8093667853001865
	Parameters used: [0.19006187 0.47381446]
Iteration

Iteration 900/2000 -- Loss Output: 0.6291469869562046 -- Moving Avg Loss: 0.7027730145826047
	Parameters used: [0.08438568 0.62752441]
Iteration 1050/2000 -- Loss Output: 0.1450346523705945 -- Moving Avg Loss: 7.628989456519101
	Parameters used: [0.3732445 0.2130644]
Iteration 1200/2000 -- Loss Output: 0.15831529249491402 -- Moving Avg Loss: 0.6977955992255588
	Parameters used: [0.20803824 0.10109745]
Iteration 1350/2000 -- Loss Output: 0.8842969964569557 -- Moving Avg Loss: 0.5502087456662865
	Parameters used: [0.08417589 0.69362912]
Iteration 1500/2000 -- Loss Output: 15.906481907510406 -- Moving Avg Loss: 9.78535550884374
	Parameters used: [0.9036477  0.49789377]
Iteration 1650/2000 -- Loss Output: 0.05590803370818251 -- Moving Avg Loss: 0.07165958710476138
	Parameters used: [0.28299844 0.5553126 ]
Iteration 1800/2000 -- Loss Output: 0.12723951772623096 -- Moving Avg Loss: 0.09903843132202393
	Parameters used: [0.24027363 0.51556034]
Iteration 1950/2000 -- Loss Output: 0.31089700385

Iteration 1800/2000 -- Loss Output: 13.856305171548543 -- Moving Avg Loss: 7.408369261874405
	Parameters used: [0.80866087 0.04322496]
Iteration 1950/2000 -- Loss Output: 0.07835034876259618 -- Moving Avg Loss: 6.560001718954381
	Parameters used: [0.13195418 0.9957863 ]
[23:29:26] Analyzing optimization parameters for tyrrel
[23:32:25] Processing Cycloid: yaphet
Iteration 150/2000 -- Loss Output: 0.45490928140477555 -- Moving Avg Loss: 0.1793351094571277
	Parameters used: [0.03041415 0.37970675]
Iteration 300/2000 -- Loss Output: 0.2962319208277576 -- Moving Avg Loss: 3.5993220303398057
	Parameters used: [0.26962529 0.34629317]
Iteration 450/2000 -- Loss Output: 0.5382111312592969 -- Moving Avg Loss: 0.5100033519830963
	Parameters used: [0.91296258 0.22189196]
Iteration 600/2000 -- Loss Output: 0.0888146961851132 -- Moving Avg Loss: 0.26678914784949403
	Parameters used: [0.1044208  0.33290053]
Iteration 750/2000 -- Loss Output: 0.04532891928308313 -- Moving Avg Loss: 0.0541307688799915

## Non-Optimized Fits 

* 0.25 Deg Obliquity
* Phases 0, 60, 120, 180, 240 and 300 degrees

In [ ]:
losses = []
folder = './output/lockedFits/'

OBLIQUITY = 0.25
phases = [0, 60, 120, 180, 240, 300]

for current in cycloids:
    for phase in phases:
        print(f'Processing phase {phase}')
        data, loss = direct_fit(cycloids[current].curve, 
                                0.25, 
                                show_plots=False, 
                                params=[phase, OBLIQUITY, 0])

        losses.append(dict(cycloid=current, loss=loss, phase=phase))
        print(current,'\t-', loss)

        filename = f'{folder}{current}-phase{phase}.csv.gz'
        data.to_csv(filename, index=False, compression='gzip')

lossFrame = pd.DataFrame(losses)
filename = f'{folder}Losses.csv'
lossFrame.to_csv(filename, index=False)

## Direct Fits (no phase or obliquity) for all cycloids

In [4]:
losses = []
folder = './output/directFits/'

for current in cycloids:
    data, loss = direct_fit(cycloids[current].curve, 0.25, show_plots=False)

    losses.append(dict(cycloid=current, loss=loss))
    print(current,'\t-', loss)

    filename = f'{folder}{current}.csv.gz'
    data.to_csv(filename, index=False, compression='gzip')

lossFrame = pd.DataFrame(losses)
filename = f'{folder}directLosses.csv'
lossFrame.to_csv(filename, index=False)

alex 	- 0.9067549953886401
carly 	- 0.4679169956204059
cilicia 	- 0.011850136742198833
delphi 	- 0.01290892547457408
dirk 	- 2.138607016165488
mira 	- 2.261342358536797
odessa 	- 0.6678619449895173
sidon 	- 0.010174794559418741
tyrrel 	- 0.8524575485454636
yaphet 	- 0.21000699087411237


## Fits with Tight Obliquity

In [ ]:
for cycloid in cycloids:
    process_cycloid_top_fits(cycloids[cycloid].curve,
                             cycloid,
                             folder='./output/tightObliquity/',
                             iterations=2000,
                             constraints=tight_obliquity_contstraints,
                             number_of_fits=5,
                             paramCount=2,
                             verbose=False
                            )

## Fits with Specific Obliquity

In [9]:
OBLIQUITY = 0.25
constraints = getConstraints(OBLIQUITY, longitude_max=0)

# for cycloid in cycloids:
#     process_cycloid_top_fits(cycloids[cycloid].curve,
#                              cycloid,
#                              folder='./output/lockedObliquity025/',
#                              iterations=2000,
#                              constraints=constraints,
#                              number_of_fits=5,
#                              paramCount=2,
#                              verbose=False
#                             )
    
process_cycloid_top_fits(cycloids['alex'].curve,
                         cycloid,
                         folder='./output/lockedObliquity025/',
                         iterations=2000,
                         constraints=constraints,
                         number_of_fits=5,
                         paramCount=2,
                         verbose=False
                        )

Iteration 150/2000 -- Loss Output: 0.045085528359826064 -- Moving Avg Loss: 1.0141615043161445
	Parameters used: [0.61336428 0.25      ]
Iteration 300/2000 -- Loss Output: 3.83810488151686 -- Moving Avg Loss: 5.755847847126807
	Parameters used: [0.99299302 0.25      ]
Iteration 450/2000 -- Loss Output: 0.06432383796660107 -- Moving Avg Loss: 0.9969965000347493
	Parameters used: [0.61486314 0.25      ]
Iteration 600/2000 -- Loss Output: 6.835573290868268 -- Moving Avg Loss: 6.913571022631994
	Parameters used: [0.1386872 0.25     ]
Iteration 750/2000 -- Loss Output: 3.1166830075450798 -- Moving Avg Loss: 4.732834950690372
	Parameters used: [0.88016518 0.25      ]
Iteration 900/2000 -- Loss Output: 1.5267607937551242 -- Moving Avg Loss: 1.056237654341831
	Parameters used: [0.67355772 0.25      ]
Iteration 1050/2000 -- Loss Output: 5.804567603047678 -- Moving Avg Loss: 8.155339591377528
	Parameters used: [0.98046102 0.25      ]
Iteration 1200/2000 -- Loss Output: 0.06322106295710206 -- Mov